In [100]:
import pandas as pd
import numpy as np

In [119]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [120]:
X_train = train.drop('Survived', axis=1)
Y_train = train['Survived']

# remove = ['Pclass', 'SibSp', 'Parch', 'Cabin' , 'Embarked', 'Fare', 'Ticket', 'Name']
X_train = train.drop(remove, axis=1)
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].median())
X_train['Sex'].replace('Female', '0')
print(X_train)

In [122]:
X_train = X_train.to_numpy()

In [123]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*32, 32*4)  # 6*6 from image dimension
        self.fc2 = nn.Linear(32*4, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [124]:
print(X_train[1])
#print(net(test_in))

[2 1 'female' 38.0]


In [98]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.001,momentum=0.9)
criterion = nn.BCELoss()
BS = 4

losses, accuracies = [], []
for i in range(2):
    samp = np.random.randint(0, X_train.shape[0], size=(BS))
    X = torch.tensor(X_train[samp]).float()
    labels = torch.tensor(Y_train[samp]).long()
    
    optimizer.zero_grad()   # zero the gradient buffers
    outputs = net(X)
    cat = torch.argmax(outputs, dim=1)
    
    acc = (labels == cat).float().mean()
    loss = criterion(outputs, labels)
    loss = loss.mean()
    losses.append(loss.item())
    
    loss.backward()
    optimizer.step()
    
    accuracy = acc.item()
    accuracies.append(acc)
plt.ylim(-0.1, 1.5)
plot(losses)
plot(accuracies)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.